In [1]:
import re
import json

In [2]:
from app.uzt_client import UZTClient
from app import pydantic_models as pd
import main

In [3]:
#getting categories list
def get_categories_from_page(client):
    categories = list()
    a_list = client.tree.css("#ctl00_MainArea_UpdatePanel1 li a")
    for a in a_list:
        category = pd.Category(
            name = a.text(),
            href = a.attrs.get('href')
        )
        categories.append(category)
    
    return categories
    

In [4]:
#getting href to next page
def get_next_page_href(client):
    a = client.tree.css_first("table.Pager td:last-child a")
    if not a:
        return False
    return a.attrs.get("href")


In [36]:
uzt = UZTClient()
uzt.get(url="https://portal.uzt.lt/LDBPortal/Pages/ServicesForEmployees.aspx")

<Response [200 OK]>

In [6]:
cats = main.get_categories()

In [ ]:
cats

In [ ]:
create_resp = main.send_data_to_api(endpoint="/api/categories/create", data=cats.data)

In [13]:
uzt.redirect_response.text

'1|#||4|190|pageRedirect||%2fLDBPortal%2fPages%2fPositionOpeningPublic%2fPublicOpenPositionCard.aspx%3fbranch%3dPOSearch%26pageId%3d643fbcc5-b793-4d14-9773-5b704d57344f%26itemID%3d6fd5b40c-5c63-497f-87cb-7cdadc412fb3|'

In [ ]:
uzt.response.text

In [ ]:
json.dumps(cats.data)

In [37]:
#open category
uzt.submit_asp_form(cats[0].href)
uzt.get(url=uzt.next_url)

<Response [200 OK]>

In [57]:
uzt.save_to_html("save00_card.html")

{'status': 'The file has been successfully saved as save00_card.html'}

In [41]:
next_page_href = main.get_next_page_href(uzt)
print(next_page_href)

ok=True data="javascript:__doPostBack('ctl00$MainArea$SearchResultsList$POGrid$ctl29$NextBtn','')"


In [27]:
#next page
uzt.submit_asp_form(next_page_href.data)


<Response [200 OK]>

In [ ]:
# open page 2
uzt.submit_asp_form("javascript:__doPostBack('ctl00$MainArea$SearchResultsList$POGrid$ctl29$PageBtn10','')")
uzt.save_to_html("save00_card.html")

In [ ]:
need_to_get = ("__VIEWSTATE", "__VIEWSTATEGENERATOR", "__EVENTVALIDATION")
asp_inputs = uzt.asp_form_inputs
asp_inputs

In [ ]:
uzt.get(url=uzt.next_url)

In [55]:
jobs_list = list()

In [56]:
# поиск строк и ячеек
table = uzt.tree.css_first("#ctl00_MainArea_SearchResultsList_POGrid")
tr_list = table.css("tr:not(:nth-child(-n+2)):not(:last-child)")
for tr in tr_list:
    cells = tr.css("td")
    date_from, date_to, title, company, place = [item.text().strip() for item in tr.css("td")]
    js_url = cells[2].css_first("a").attrs.get("href")
    uzt.submit_asp_form(js_url, only_url=True)
    dirty_url = uzt.next_url
    match = re.search(r"(^.+aspx\?).+(itemID.+)$", dirty_url)
    url = match.group(1)+ match.group(2)
    job = pd.Job(date_from=date_from,
              date_to= date_to,
              title= title,
              company= company,
              place= place,
              url = url,
             )
    
    jobs_list.append(job.dict())
    
print(len(jobs_list))
print(json.dumps(jobs_list))
cells
job


25
[{"date_from": "2023-05-08", "date_to": "2023-05-19", "title": "\u016akio darbininkas", "company": "\u016akininko \u016bkis, 1359663", "place": "Mol\u0117t\u0173 rajono sav., Kreivi\u0161kiai", "url": "https://portal.uzt.lt//LDBPortal/Pages/PositionOpeningPublic/PublicOpenPositionCard.aspx?itemID=b70f28b2-89e0-4608-acfd-763670ca62b6"}, {"date_from": "2023-05-08", "date_to": "2023-05-30", "title": "Ekskavatorininkas", "company": "U\u017edaroji akcin\u0117 bendrov\u0117 \"DOVYRA\"", "place": "\u0160iauli\u0173 miesto sav., \u0160iauliai", "url": "https://portal.uzt.lt//LDBPortal/Pages/PositionOpeningPublic/PublicOpenPositionCard.aspx?itemID=6fd5b40c-5c63-497f-87cb-7cdadc412fb3"}, {"date_from": "2023-05-08", "date_to": "2023-05-30", "title": "Nekvalifikuotas mel\u017e\u0117jas", "company": "\u016akininko \u016bkis, 6932788", "place": "Pasvalio rajono sav., Kiemeliai", "url": "https://portal.uzt.lt//LDBPortal/Pages/PositionOpeningPublic/PublicOpenPositionCard.aspx?itemID=06bf6afc-f587-4

Job(date_from='2023-05-08', date_to='2023-06-30', title='Gyvulių ir fermos prižiūrėtojos', company='Uždaroji akcinė bendrovė Dotnuvos eksperimentinis ūkis', place='Kėdainių rajono sav., Valinava', url='https://portal.uzt.lt//LDBPortal/Pages/PositionOpeningPublic/PublicOpenPositionCard.aspx?itemID=ea942407-cc68-418a-975e-724b6ce60cae')

In [25]:
# open job's card
uzt.submit_asp_form("javascript:__doPostBack('ctl00$MainArea$SearchResultsList$POGrid$ctl04$PositionTitleButton','')", only_url=True)

<Response [200 OK]>

In [22]:
uzt.next_url

'https://portal.uzt.lt//LDBPortal/Pages/PositionOpeningPublic/PublicOpenPositionCard.aspx?branch=POSearch&pageId=3cd162b2-1ca2-4282-8514-f69eda327a50&itemID=87b2785d-857c-4b7f-b538-7d75e2982291'

In [ ]:
uzt.response.text

In [ ]:
uzt.asp_form_inputs
